# 2. Deep dive into a BSDF

## Overview

As you will soon discover, Mitsuba 2 can do much more than rendering. In this tutorial we will show how to instanciate a BSDF plugin using Python dictionaries and plot inspect its distribution function using `matplotlib` and `numpy`.

<div class="alert alert-block alert-info">

✔️ **What will you learn?**

<ul>
  <li>Instanciate Mitsuba objects using Python dict and mitsuba.core.load_dict</li>
  <li>Perform vectorized computations using a JIT variant of Mitsuba</li>
  <li>Convert an enoki array into a numpy array</li>
</ul>
    
</div>


### Step 1 - setup

Of course, let's start with the usual Python imports! As emphasized in the previous tutorial, Mitsuba requires a specific variant to be set before performing any other imports or computations. For this tutorial, we are going to use one of the JIT vectorized variant of the system. This will allow us to write code as if it was operating on normal scalar values, and have it run on arbitrary-sized arrays of values on the CPU or GPU.

In [1]:
# Add mitsuba and enoki to PATH (this shouldn't be necessary)
import sys
sys.path.append('../../../build/python')

In [2]:
import enoki as ek
import mitsuba

# Set the desired mitsuba JIT variant
mitsuba.set_variant('llvm_ad_rgb')

from mitsuba.core import load_dict, Float, Vector3f
from mitsuba.render import SurfaceInteraction3f, BSDFContext

### Step 2 - instanciate a BSDF

One easy way to instanciate Mitsuba objects (e.g. [Shape](../generated/plugins_shapes.html#sec-shapes), [BSDF](../generated/plugins_bsdfs.html#sec-bsdfs), ...) is using the [<code>mitsuba.core.load_dict</code>][1] function. This function takes as input a Python `dict` following a similar structure to the XML scene description and instantiates the corresponding plugin. In our scenario, we want to construct a [roughconductor BSDF](../generated/plugins_bsdfs.html#bsdf-roughconductor) with a high roughness value and a GGX microfacet distribution.

Refer to the [documentation](../src/python_interface/parsing_xml.html#creating-objects-using-python-dictionaries) for more information regarding the Python `dict` convention for [<code>mitsuba.core.load_dict</code>][1]



[1]: https://mitsuba2.readthedocs.io/en/latest/generated/core_api.html?highlight=load_dict#mitsuba.core.load_dict

In [3]:
bsdf = load_dict({
    'type': 'roughconductor',
    'alpha': 0.2,
    'distribution': 'ggx'
})

### Step 3 - vectorized evaluation the BSDF

We will now evaluate this BSDF for a whole array of directions at once, leveraging the enabled vectorize backend.

In [4]:
def sph_dir(theta, phi):
    """ Map spherical to Euclidean coordinates """
    st, ct = ek.sincos(theta)
    sp, cp = ek.sincos(phi)
    return Vector3f(cp * st, sp * st, ct)

# Create a (dummy) surface interaction to use for the evaluation
si = ek.zero(SurfaceInteraction3f)

# Specify an incident direction with 45 degrees elevation
si.wi = sph_dir(ek.Pi * 45 / 180, 0.0)

# Create grid in spherical coordinates and map it onto the sphere
res = 300
theta_o, phi_o = ek.meshgrid(
    ek.linspace(Float, 0,     ek.Pi,     res),
    ek.linspace(Float, 0, 2 * ek.Pi, 2 * res)
)
wo = sph_dir(theta_o, phi_o)

# Evaluate the whole array (18000 directions) at once
values = bsdf.eval(BSDFContext(), si, wo)

### Step 4 - convert and plot the results

Enoki arrays of any flavour can easily be converted to array type of other mainstream libraries, such as `numpy`, `PyTorch`, `JAX` and `TensorFlow`. For more detailed information on this, take a look at the extensive [enoki documentation](https://enoki.readthedocs.io/en/master/).
In our case we are going to convert our enoki array into a numpy array.

In [5]:
import numpy as np
values_np = np.array(values)

We can now use our favourite plotting library to visualize the BSDF distribution (here we use `matplotlib`).

In [6]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Extract red channel of BRDF values and reshape into 2D grid
values_r = values_np[:, 0]
values_r = values_r.reshape(2 * res, res).T

# Plot values for spherical coordinates
fig, ax = plt.subplots(figsize=(8, 4))
im = ax.imshow(values_r, extent=[0, 2 * np.pi, np.pi, 0], cmap='jet')
ax.set_xlabel(r'$\phi_o$', size=10)
ax.set_xticks([0, np.pi, 2 * np.pi])
ax.set_xticklabels(['0', '$\\pi$', '$2\\pi$'])
ax.set_ylabel(r'$\theta_o$', size=10)
ax.set_yticks([0, np.pi / 2, np.pi])
ax.set_yticklabels(['0', '$\\pi/2$', '$\\pi$'])
cax = make_axes_locatable(ax).append_axes("right", size="3%", pad=0.05)
plt.colorbar(im, cax=cax);

### *Reference API*

- [<code>enoki.zero</code>](...)
- [<code>enoki.linspace</code>](...)
- [<code>enoki.meshgrid</code>](...)
- [<code>enoki.sincos</code>](...)
- [<code>mitsuba.core.load_dict</code>](...)
- [<code>mitsuba.render.SurfaceInteraction3f</code>](...)
- [<code>mitsuba.render.BSDFContext</code>](...)
- [<code>mitsuba.render.BSDF.eval</code>](...)

---
<div class="alert alert-block alert-info" 
     style="cursor: pointer;" 
     onclick="window.location='https://github.com/mitsuba-renderer/mitsuba2/';">
📑 Checkout full Jupyter notebook 📑
</div>